In [ ]:
# import packages
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing,metrics 
from haversine import haversine
from datetime import datetime
import xgboost as xgb
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import warnings
from sklearn import linear_model
from sklearn.model_selection import train_test_split,KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression,Lasso,LinearRegression, Ridge,BayesianRidge
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.cluster import KMeans 
from sklearn.svm import SVR
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 100)
from geopy import distance
%matplotlib inline
from scipy.spatial import distance as scidist

In [ ]:
# load training, weather and holidays data 
train_data = pd.read_csv('train.csv')
wd = pd.read_csv('weather.csv')
holidays_data = pd.read_csv('holidays.csv')

In [ ]:
# Compute is_holiday
def compute_holiday(row):
    if row['day_of_year'] in holidays_data['day_of_year']:
        return 1
    return 0

In [ ]:
# weather features: merge with train_data
wd = wd.replace({"precipitation": {"T":0.01}, "snow fall": {"T":0.01}, "snow depth": {"T":0.1}})

wd['date'] = pd.to_datetime(wd['date'], format='%d-%m-%Y')
wd['day_of_year'] = wd['date'].dt.dayofyear

holidays_data['date'] = pd.to_datetime(holidays_data['date'])
holidays_data['day_of_year'] = holidays_data['date'].dt.dayofyear

train_data['pickup_datetime'] = pd.to_datetime(train_data['pickup_datetime'])
train_data['day_of_year'] = train_data['pickup_datetime'].dt.dayofyear
train_data = pd.merge(train_data, wd, on='day_of_year')

train_data['is_holiday'] = train_data.apply(compute_holiday, axis=1)

In [ ]:
# rush hours features
# Add is_weekend ? is_work_hour ? features.
train_data['pickup_datetime'] = pd.to_datetime(train_data['pickup_datetime']) 
train_data['dropoff_datetime'] = pd.to_datetime(train_data['dropoff_datetime']) 

train_data['pickup_hours'] = train_data['pickup_datetime'].dt.hour
train_data['pickup_day'] = train_data['pickup_datetime'].dt.weekday
train_data['dropoff_hours'] = train_data['dropoff_datetime'].dt.hour

weekends = np.where(train_data['pickup_day'] > 4, 1, 0)
train_data['is_weekend'] = weekends

lower_hrs = np.where(train_data['pickup_hours'] > 7, 1, 0)
upper_hrs = np.where(train_data['dropoff_hours'] < 9, 1, 0)
work_hours = np.where(lower_hrs == upper_hrs, lower_hrs, 0)
train_data['is_rush_hour_am'] = work_hours

lower_hrs = np.where(train_data['pickup_hours'] > 16, 1, 0)
upper_hrs = np.where(train_data['dropoff_hours'] < 18, 1, 0)
work_hours = np.where(lower_hrs == upper_hrs, lower_hrs, 0)
train_data['is_rush_hour_pm'] = work_hours

# drop all temps created
train_data.drop(labels=['pickup_day'], axis=1, inplace=True)

train_data.head()

In [ ]:
#drop uneceessary features
train_data.drop(labels=['id','pickup_datetime','dropoff_datetime', 
                        'date'], axis=1, inplace=True)

train_data.head()

In [ ]:
# check for missing column values and check data shape
missing_train = train_data.isnull().mean().sort_values(ascending=False)
print(missing_train.head(5))
print(train_data.shape)

# Additional Features Choice 
- Distance to Empire State Building ::
located on East 34th Street between 5th Ave. and FDR Dr., one of the most popular corridors for traffic in North America ==> Empire State Building Location: (40.748940, -73.985625)
https://patch.com/new-york/new-york-city/nyc-worlds-third-worst-city-traffic-congestion
The location is also one of the most visited in NYC for tourism: https://www.planetware.com/tourist-attractions-/new-york-city-us-ny-nyc.htm

- Distance to Bryant Park ::
located on East 42nd Street which is also mentioned in the article as one of the busiest spots. Location is also a famous touristic area right before FDR Dr. ==> Bryant: (40.753318, -73.982734)
https://patch.com/new-york/new-york-city/nyc-worlds-third-worst-city-traffic-congestion

- Distance to Financial District ::
lot of people commute there because of work and business. During rush hours, expecting significant number of people trying to go there thereby increasing the trip duration ==> FD (40.707697, -74.008291)

- Times Square ==> (40.758901, -73.985136) Some of the most visited places

In [ ]:
# Helper functions for distance calculations
def fromPickUpHelper(row, lat, lon):
    u = [row['pickup_latitude'], row['pickup_longitude']]
    v = [lat, lon]
    return scidist.euclidean(u, v)
#   return haversine((row['pickup_latitude'], row['pickup_longitude']), (lat, lon))

def toDestinationHelper(row, lat, lon):
    u = [row['dropoff_latitude'], row['dropoff_longitude']]
    v = [lat, lon]
    return scidist.euclidean(u, v)
#   return haversine((row['dropoff_latitude'], row['dropoff_longitude']), (lat, lon))

def euclideanDist(row):
    u = [row['pickup_latitude'], row['pickup_longitude']]
    v = [row['dropoff_latitude'], row['dropoff_longitude']]
    return scidist.euclidean(u, v)

def manhattanDist(row):
    u = [row['pickup_latitude'], row['pickup_longitude']]
    v = [row['dropoff_latitude'], row['dropoff_longitude']]
    return scidist.cityblock(u, v)

In [ ]:
# Euclidean Distance
train_data['euclidean_dist'] = train_data.apply(euclideanDist, axis=1)

In [ ]:
# Manhattan Distance
train_data['manhattan_dist'] = train_data.apply(manhattanDist, axis=1)

In [ ]:
train_data.sample(5)

In [ ]:
# geodesic distance feature
td_geodes = [] 
train_dist = []

for index, row in train_data.iterrows():
    dist_train = haversine([row['pickup_latitude'],row['pickup_longitude']],[row['dropoff_latitude'],row['dropoff_longitude']])
    geodesic_dist = distance.distance((row['pickup_latitude'], row['pickup_longitude']), (row['dropoff_latitude'], row['dropoff_longitude'])).km

    train_dist.append(dist_train)
    td_geodes.append(geodesic_dist)
train_data['geodesic_dist'] = td_geodes
train_data['haversine_dist'] = train_dist


In [ ]:
#analyze and remove outliers
max_dist_train = np.max([train_data['geodesic_dist']])
print("Max_dist", max_dist_train)
min_dist_train = np.min([train_data['geodesic_dist']])
print("Min_dist",min_dist_train)
mean_dist_train = np.mean([train_data['geodesic_dist']])
print("Mean_dist",mean_dist_train)
train_data['geodesic_dist'].describe()
dist_std_train = np.std(train_data['geodesic_dist'])
train_data = train_data[train_data['geodesic_dist'] <= mean_dist_train + 2*dist_std_train]
train_data = train_data[train_data['geodesic_dist'] >= mean_dist_train - 2*dist_std_train]
train_data['geodesic_dist'].describe()


In [ ]:
# Compute distance of pickup to busy areas in Manhattan
fd = (40.707697, -74.008291) #financial district
bryant_park= (40.753318, -73.982734)
times_sq = (40.758901, -73.985136) 
ctrl_park = (40.783043, -73.965244)
empire_state = (40.748940, -73.985625)
memorial_911 = (40.711495, -74.013440)

train_data['pickup_dist_FD'] = train_data.apply(fromPickUpHelper, args=fd, axis=1)
train_data['pickup_dist_bryant_park'] = train_data.apply(fromPickUpHelper, args=bryant_park, axis=1)
train_data['pickup_dist_times_square'] = train_data.apply(fromPickUpHelper, args=times_sq, axis=1)
train_data['pickup_dist_central_park'] = train_data.apply(fromPickUpHelper, args=ctrl_park, axis=1)
train_data['pickup_dist_empire_state'] = train_data.apply(fromPickUpHelper, args=empire_state, axis=1)
train_data['pickup_dist_memorial_911'] = train_data.apply(fromPickUpHelper, args=memorial_911, axis=1)

In [ ]:
# Compute distance of destination to busy areas in Manhattan
train_data['dropoff_dist_FD'] = train_data.apply(toDestinationHelper, args=fd, axis=1)
train_data['dropoff_dist_bryant_park'] = train_data.apply(toDestinationHelper, args=bryant_park, axis=1)
train_data['dropoff_dist_times_square'] = train_data.apply(toDestinationHelper, args=times_sq, axis=1)
train_data['dropoff_dist_central_park'] = train_data.apply(toDestinationHelper, args=ctrl_park, axis=1)
train_data['dropoff_dist_empire_state'] = train_data.apply(toDestinationHelper, args=empire_state, axis=1)
train_data['dropoff_dist_memorial_911'] = train_data.apply(toDestinationHelper, args=memorial_911, axis=1)
train_data.head()

In [ ]:
#encoding necessary features and randomly sample to reduce dataset size
train_data = train_data.replace({"store_and_fwd_flag": {"N":0, "Y":1}})
# train_data = train_data.sample(frac=0.05,random_state=4)

In [ ]:
#transforming duration to log scale to allow for RMSE plus spliting data
train_data['log_duration'] = np.log(train_data['trip_duration'].values + 1)
train_data.drop(labels=['trip_duration'], axis=1, inplace=True)
train_target = pd.DataFrame(train_data['log_duration'])
train_data.drop('log_duration', axis = 1, inplace = True)
X_train, X_test, y_train, y_test = train_test_split(np.array(train_data), np.array(train_target), test_size=0.30)
eval_set=[(X_test, y_test)]
print("train_target: ", train_target.shape)
print('train_set: ', X_train.shape, y_train.shape)
print('test_set: ', X_test.shape, y_test.shape)

In [ ]:
# Visualization of lon, lat
fig, ax = plt.subplots(ncols=1, nrows=1,figsize=(12,10))
plt.ylim(40.5, 41)
plt.xlim(-74.1, -73.7)
ax.set_xlabel('longitude')
ax.set_ylabel('latitude')
ax.set_title('Scatter plot of pickup locations in New York City')
ax.scatter(train_data['pickup_longitude'],train_data['pickup_latitude'], s=0.1, color='black')

fig, ax = plt.subplots(ncols=1, nrows=1,figsize=(12,10))
plt.ylim(40.5, 40.9)
plt.xlim(-74.2, -73.6)
ax.set_xlabel('longitude')
ax.set_ylabel('latitude')
ax.set_title('Scatter plot of dropoff locations in New York City')
ax.scatter(train_data['dropoff_longitude'],train_data['dropoff_latitude'], s=0.1, color='black')

In [ ]:
#Random Forest
random_forest_regressor= RandomForestRegressor(n_estimators = 100, n_jobs = 4)
random_forest_regressor.fit(X_train, y_train)
rf_y_pred = random_forest_regressor.predict(X_test)
random_forest_error =  np.sqrt(mean_squared_error(y_test, rf_y_pred))
print("Test error: ", random_forest_error)
print(y_test[0:10])
print(rf_y_pred[0:10])

In [ ]:
0#Feature importance
fig, ax = plt.subplots()
width = 0.6
feature_importances = pd.DataFrame(random_forest_regressor.feature_importances_, index = train_data.columns, columns = ['Importance']).sort_values('Importance', ascending = False)
print(feature_importances)
ax.bar(np.arange(len(train_data.columns)), random_forest_regressor.feature_importances_, width, color='b')
ax.set_xticks(np.arange(len(random_forest_regressor.feature_importances_)))
ax.set_xticklabels(train_data.columns.values, rotation = 90, horizontalalignment='right')
plt.title('Feature Importances')
ax.set_ylabel('Normalized Gini Importance')

In [ ]:
#XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtrain.save_binary('train.buffer')
dtest = xgb.DMatrix(X_test, label=y_test)
dtest.save_binary('test.buffer')
evallist = [(dtest, 'eval'), (dtrain, 'train')]
param = {'min_child_weight': 3, 'eta': 0.9, 'colsample_bytree': 0.6, 'max_depth': 6,
            'subsample': 0.9, 'lambda': 1., 'nthread': -1,'booster' : 'gbtree',
            'eval_metric': 'rmse', 'objective': 'reg:linear'}
bst = xgb.train(param, dtrain, 100, evallist, early_stopping_rounds=2, maximize=False, verbose_eval=1)
print('Modeling RMSLE %.5f' % bst.best_score)
pred = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)
pred = np.exp(pred) - 1

In [ ]:
#Multi-layer perceptron
multi_layer_perceptron_regressor = MLPRegressor(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes = (5, 2), random_state = 1)
multi_layer_perceptron_regressor.fit(X_train, y_train)     
mlp_y_pred = multi_layer_perceptron_regressor.predict(X_test)
mlp_error = np.sqrt(mean_squared_error(y_test, mlp_y_pred))
print("Test error: ", mlp_error)

In [ ]:
#Multi-layer perceptron with Adam solver + changed hyperparams
multi_layer_perceptron_regressor_2 = MLPRegressor(hidden_layer_sizes=(200,), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum = True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08,n_iter_no_change = 10)
multi_layer_perceptron_regressor_2.fit(X_train, y_train)     
mlp_y_pred_2 = multi_layer_perceptron_regressor_2.predict(X_test)
mlp_error_2 = np.sqrt(mean_squared_error(y_test, mlp_y_pred_2))
print("Test error: ", mlp_error_2)

In [ ]:
#svm using various kernels
svm_kernel_err = []
kernels = ['rbf']
for curr_kernel in kernels:
    print('Running...', curr_kernel)
    if curr_kernel == 'poly':
        model_svm = SVR(kernel= curr_kernel, degree=3)
    else:
        model_svm = SVR(kernel= curr_kernel)
    model_svm.fit(X_train, y_train)
    svm_pred = model_svm.predict(X_test)
    svm_pred_error = np.sqrt(mean_squared_error(y_test, svm_pred))
    print(curr_kernel)
    print("Test error: ", svm_pred_error)
    svm_kernel_err.append(svm_pred_error)

In [ ]:
err_list = [random_forest_error, bst.best_score, mlp_error_2, svm_kernel_err[-1]]
x = np.arange(4)
plt.bar(x, height=err_list, color='green')
plt.xticks(x, ['random_forrest', 'xgb', 'mlp_error_2', 'gaussian_svm'])
plt.ylabel('RMSLE score')